In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [2]:
url_tab = 'https://www.resetera.com/threads/media-create-sales-week-53-2020-dec-28-jan-03.356956/'


In [3]:
def obtain_table(url):
    ustitles = requests.get(url)
    soup = BeautifulSoup(ustitles.text)
    aa=soup.findAll("div", {"class": "message-cell message-cell--main"})
    return aa[0].findAll("code")[-1].text

In [4]:
text_table=obtain_table(url_tab)

In [5]:
print(text_table)

+-------+------------+------------+------------+------------+------------+-------------+
|System |  This Week |  Last Week |  Last Year |     YTD    |  Last YTD  |     LTD     |
+-------+------------+------------+------------+------------+------------+-------------+
| NSW # |    312.121 |    278.173 |    284.827 |    312.121 |    284.827 |  17.652.495 |
| PS4 # |     17.876 |     12.718 |     58.244 |     17.876 |     58.244 |   9.308.766 |
| PS5 # |     10.632 |     13.188 |            |     10.632 |            |     265.782 |
| 3DS # |        714 |        658 |      4.030 |        714 |      4.030 |  24.559.622 |
| XBS # |        134 |        205 |            |        134 |            |      31.558 |
+-------+------------+------------+------------+------------+------------+-------------+
|  ALL  |    341.477 |    304.942 |    347.213 |    341.477 |    347.213 |  51.818.223 |
+-------+------------+------------+------------+------------+------------+-------------+
|  PS5  |      7.606 

In [14]:
def tab_to_df(text_tab,year,week,days):
    tt=[]
    
    # These for bucles creates a nested list with the table
    for element in text_tab.split("+\n"):
        for element2 in element.split("\n"):
            el2=[]
            for el in element2.split("|"):
                if not((el[:1]=="\n") or (el=="") or (el[0]=="+")):
                    text_el=el.replace(".","").replace("#","") # Formating the strings
                    el2.append(text_el)
            if el2 != []:
                tt.append(el2)

    for j in range(0,len(tt[0])):
        # Remove extra spaces in the header
        tt[0][j]=tt[0][j].lstrip().rstrip()
   
    for i in range(1,len(tt)):
        # Remove extra spaces in the name of the system
        tt[i][0]=tt[i][0].lstrip().rstrip() 
        for j in range(1,len(tt[0])):
            # For each cell convert the sales into an integer. 
            # If the cell is empty, avoid the error and put a 0 instead
            try:
                tt[i][j]=int(tt[i][j])
            except ValueError:
                tt[i][j]=0

    df = pd.DataFrame(tt[1:],columns=tt[0])
    
    df["Year"]=year
    df["Week"]=week
    df["Days"]=days
    return df

In [15]:
# https://www.neogaf.com/threads/media-create-sales-archive-thread-2.416253/#post-147729422

In [16]:
url="https://www.resetera.com/threads/media-create-sales-archive-thread.4115/"
ustitles = requests.get(url)
soup = BeautifulSoup(ustitles.text)
links=soup.findAll("a", {"class": "link link--internal"})  #class="link link--internal" 

In [17]:
links_clean=[]
links_data=[]
df = pd.DataFrame()
error=[]
for el in links[::-1]:
    if ( el.get('href').find("new-used") < 0):
        links_clean.append(el.get('href'))
        try:
            text_tab=obtain_table(links_clean[-1])

            temp=links_clean[-1].split(".")[2].split("-")[-6:]
            links_data.append([int(temp[1]),int(temp[0]),temp[2]+" "+temp[3]+" / "+temp[4]+" "+temp[5]])
            df=df.append(tab_to_df(text_tab,int(temp[1]),int(temp[0]),temp[2]+" "+temp[3]+" / "+temp[4]+" "+temp[5]))
        except:
            error.append(links_clean[-1])

df.reset_index()



,index,System,This Week,Last Week,Last Year,YTD,Last YTD,LTD,Year,Week,Days
0,0,NSW,278173,263304,234268,5956943,4493885,17340374.0,2020,52,dec 21 / dec 27
1,1,PS5,13188,17578,0,255150,0,255150.0,2020,52,dec 21 / dec 27
2,2,PS4,12718,10343,81349,542647,1196153,9290890.0,2020,52,dec 21 / dec 27
3,3,3DS,658,583,3045,62761,191183,24558908.0,2020,52,dec 21 / dec 27
4,4,XBS,205,375,0,31424,0,31424.0,2020,52,dec 21 / dec 27
...,...,...,...,...,...,...,...,...,...,...,...
2294,10,PS4,25930,19950,26719,1204727,1157092,5009293.0,2017,42,oct 16 / oct 22
2295,11,PSV,3832,4142,7514,334911,700356,5582330.0,2017,42,oct 16 / oct 22
2296,12,n-2DSLL,8272,11563,0,344753,0,344753.0,2017,42,oct 16 / oct 22
2297,13,2DS,846,1055,8728,248057,82889,545204.0,2017,42,oct 16 / oct 22


In [18]:
error

['https://www.resetera.com/threads/media-create-sales-week-32-2019-aug-05-aug-11.134991/',
 'https://www.resetera.com/threads/media-create-sales-week-19-2019-may-06-may-12.117093/',
 'https://www.resetera.com/threads/media-create-sales-week-18-2019-apr-29-may-05.116534/',
 'https://www.resetera.com/threads/media-create-sales-week-52-2017-dec-25-dec-31.14650/']

In [19]:
df.columns

Index(['System', 'This Week', 'Last Week', 'Last Year', 'YTD', 'Last YTD',
       'LTD', 'Year', 'Week', 'Days'],
      dtype='object')

In [20]:
df

,System,This Week,Last Week,Last Year,YTD,Last YTD,LTD,Year,Week,Days
0,NSW,278173,263304,234268,5956943,4493885,17340374.0,2020,52,dec 21 / dec 27
1,PS5,13188,17578,0,255150,0,255150.0,2020,52,dec 21 / dec 27
2,PS4,12718,10343,81349,542647,1196153,9290890.0,2020,52,dec 21 / dec 27
3,3DS,658,583,3045,62761,191183,24558908.0,2020,52,dec 21 / dec 27
4,XBS,205,375,0,31424,0,31424.0,2020,52,dec 21 / dec 27
...,...,...,...,...,...,...,...,...,...,...
10,PS4,25930,19950,26719,1204727,1157092,5009293.0,2017,42,oct 16 / oct 22
11,PSV,3832,4142,7514,334911,700356,5582330.0,2017,42,oct 16 / oct 22
12,n-2DSLL,8272,11563,0,344753,0,344753.0,2017,42,oct 16 / oct 22
13,2DS,846,1055,8728,248057,82889,545204.0,2017,42,oct 16 / oct 22


In [29]:
df[df["System"]=="NSW"]["This Week"].max()

392576

In [34]:
df[df["This Week"] == df[df["System"]=="NSW"]["This Week"].max()]

,System,This Week,Last Week,Last Year,YTD,Last YTD,LTD,Year,Week,Days
0,NSW,392576,57274,56812,1417791,919371,12801222.0,2020,12,mar 16 / mar 22
